# 상품·리뷰 → 카테고리 리포트 + Qwen 7B Colab 학습

- **커널:** 우측 상단 **Select Kernel** → **Colab** 선택 후 **GPU** 런타임 사용
- Google Drive `tips_ai_colab` 폴더에 **필수 파일** 업로드 후 아래 셀을 순서대로 실행
- **4번 학습** 끝나면 반드시 **4-1 Drive 복사** 셀 실행 (로컬 → Drive 복사). 안 하면 런타임 끊길 때 결과 날아감.
- **자동 실행 (한 셀):** **`colab_run_all.ipynb`** — 코드 셀 하나만 실행하면 1→2→3→4→4-1 자동 진행.  
- **정확한 명령어·오류 시 쳐야 할 것:** **`COLAB_RUN_ORDER.md`** (복붙용 순서 + 오류별 조치)  
- 자세한 흐름·CSV 형식·트러블슈팅: **`CATEGORY_REPORT_COLAB_7B.md`** 참고

**필수:** `22_prepare_report_summary_sft.py`, `23_train_report_summary_lora.py`, `report_summary_lib.py`, `ai_report_bullets_lib.py`, `training_report_summary_sft_500.jsonl` (또는 22로 생성한 JSONL)  
**선택:** `24_generate_report_summary.py`, `25_generate_category_report_from_csv.py`, `products_all.csv`, `reviews_all.csv` (SFT·리포트 생성 시)

**DB에서 CSV 가져오기:** 로컬에서 `.env` 설정 후 `python 21_export_products_reviews_from_db.py` 실행 → `products_all.csv`, `reviews_all.csv` 생성 → `tips_ai_colab`에 업로드. (`env.example` 참고)

In [1]:
# 1. Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: 

In [ ]:
# 2. 작업 디렉터리 + 파일 복사
import os
os.makedirs('/content/tips_ai', exist_ok=True)
%cd /content/tips_ai
!cp -r /content/drive/MyDrive/tips_ai_colab/* /content/tips_ai/
!ls -lh

In [ ]:
# 3. 패키지 설치 + GPU 확인
!pip install -q transformers peft datasets accelerate torch bitsandbytes pandas
import torch
print('GPU:', torch.cuda.is_available(), torch.cuda.get_device_name(0) if torch.cuda.is_available() else '')

---

### (선택) 3-1. SFT 학습 데이터 생성 (22)

`products_all.csv`, `reviews_all.csv`가 `tips_ai_colab`에 있으면 실행.  
출력 `training_report_summary_sft.jsonl`을 4번 학습에 쓰려면 아래 4번 셀에서 `--train_jsonl training_report_summary_sft.jsonl`로 바꿔서 실행하세요.

In [ ]:
# (선택) SFT JSONL 생성 — products_all.csv, reviews_all.csv 필요
# %cd /content/tips_ai
# !python 22_prepare_report_summary_sft.py \
#   --input_csv products_all.csv \
#   --reviews_csv reviews_all.csv \
#   --out_jsonl training_report_summary_sft.jsonl \
#   --samples_per_category 50 \
#   --min_products 10
# !head -n 1 training_report_summary_sft.jsonl

In [ ]:
# 4. LoRA 학습 실행 (Qwen 7B)
# 반드시 1번(Drive 마운트) → 2번(파일 복사) → 3번(패키지) 실행 후 진행.
%cd /content/tips_ai
!ls -la 23_train_report_summary_lora.py training_report_summary_sft_500.jsonl
# 22 실행 후 JSONL 쓴 경우: --train_jsonl training_report_summary_sft.jsonl 로 변경
# out_dir 은 로컬 저장. 4-1에서 Drive로 복사.
!python 23_train_report_summary_lora.py \
  --base_model Qwen/Qwen2.5-7B-Instruct \
  --train_jsonl training_report_summary_sft_500.jsonl \
  --out_dir /content/tips_ai/results/qwen2.5-7b-lora-report-summary \
  --epochs 5 --lr 1e-4 --max_length 2048 \
  --lora_r 8 --lora_alpha 16 --batch_size 1 --grad_accum 8 \
  --save_steps 100 --logging_steps 10

### 4-1. 학습 결과 → Drive 복사 (필수)

학습은 **로컬**(`/content/tips_ai/results/`)에 저장됨. 여기서 **Drive**로 복사해야 런타임 끊겨도 유지됨.

In [ ]:
# 로컬 결과 확인 후 Drive로 복사
!ls -la /content/tips_ai/results/qwen2.5-7b-lora-report-summary
!mkdir -p /content/drive/MyDrive/tips_ai_colab/results
!cp -r /content/tips_ai/results/qwen2.5-7b-lora-report-summary /content/drive/MyDrive/tips_ai_colab/results/
!ls -la /content/drive/MyDrive/tips_ai_colab/results/qwen2.5-7b-lora-report-summary
print("\n✅ Drive 저장 완료: 내 드라이브 → tips_ai_colab → results → qwen2.5-7b-lora-report-summary")

---

### (선택) 5. 카테고리 리포트 생성 — 템플릿 기반 (25)

`products_all.csv`, `reviews_all.csv`가 있으면 특정 카테고리 기준으로 리포트 JSON 생성.  
아래 `--category_contains`를 원하는 카테고리 문자열로 바꿔서 실행하세요.

In [ ]:
# (선택) 템플릿 기반 카테고리 리포트 JSON 생성
# !python 25_generate_category_report_from_csv.py \
#   --products_csv products_all.csv \
#   --reviews_csv reviews_all.csv \
#   --category_contains "유모차" \
#   --out_json /content/drive/MyDrive/tips_ai_colab/report_category.json
# !head -c 500 /content/drive/MyDrive/tips_ai_colab/report_category.json

### (선택) 6. 모델 기반 리포트 생성 — 7B LoRA + CSV (24)

학습이 끝난 뒤, CSV로 **학습된 7B**를 써서 요약 생성.  
`--model_dir`는 4번 `--out_dir`과 동일하게 맞추세요.

In [ ]:
# (선택) 학습된 7B LoRA + CSV로 요약 생성
# !python 24_generate_report_summary.py \
#   --model_dir /content/drive/MyDrive/tips_ai_colab/results/qwen2.5-7b-lora-report-summary \
#   --products_csv products_all.csv \
#   --reviews_csv reviews_all.csv \
#   --category_contains "유모차"